In [ ]:
# Here we will create DynamoDB table 'job_run_details' to capture each and every jobs run details,
#because will be executed multiple times

In [1]:
import boto3

In [2]:
import os
os.environ.setdefault('AWS_PROFILE', 'gitawsacct')

'gitawsacct'

In [3]:
dynamo_client = boto3.client('dynamodb')

In [4]:
type(dynamo_client)

botocore.client.DynamoDB

In [5]:
dynamo_client.list_tables()

{'TableNames': ['jobs'],
 'ResponseMetadata': {'RequestId': '7VL9BGTNVUIOSD8UDFKMMGEI4BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Apr 2023 20:00:09 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '23',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7VL9BGTNVUIOSD8UDFKMMGEI4BVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '312962171'},
  'RetryAttempts': 0}}

In [6]:
tables = dynamo_client.list_tables()['TableNames']

In [7]:
'jobs' in tables

False

In [ ]:
#to delete existing table
dynamo_client.delete_table(Table_name='job_run_details')

In [8]:
#create table with composite key , job id(S -string)itself won't be unique it has to be associated with 
#job run time(N - number), Unix Epoch will be stored as job run time. Within each job id all the jobs will be preserved
#in ascending or descending order depending upon how the data is managed internally. Hence all the data related to a
#given job id will be together and sorted based on job run time.
response = dynamo_client.create_table(
    AttributeDefinitions=[
        {
            'AttributeName': 'job_id',
            'AttributeType': 'S'
        },
            {
            'AttributeName': 'job_run_time',
            'AttributeType': 'N'
        }
    ],
    TableName='job_run_details',
    KeySchema=[
        {
            'AttributeName': 'job_id',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'job_run_time',
            'KeyType': 'RANGE'
        }
    ],
    BillingMode='PAY_PER_REQUEST'
)


In [9]:
response

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'job_id',
    'AttributeType': 'S'},
   {'AttributeName': 'job_run_time', 'AttributeType': 'N'}],
  'TableName': 'job_run_details',
  'KeySchema': [{'AttributeName': 'job_id', 'KeyType': 'HASH'},
   {'AttributeName': 'job_run_time', 'KeyType': 'RANGE'}],
  'TableStatus': 'CREATING',
  'CreationDateTime': datetime.datetime(2023, 4, 23, 15, 7, 45, 440000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:866874214817:table/job_run_details',
  'TableId': '477c032b-1fd9-4d74-b3d3-e4c0dbbb7e2a',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST'}},
 'ResponseMetadata': {'RequestId': 'AL177NIS4S055C7E0E8A2LRS2RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Apr 2023 20:07:45 GMT',
   'content-typ

In [10]:
#Validate above table
dynamo_client.list_tables()

{'TableNames': ['job_run_details', 'jobs'],
 'ResponseMetadata': {'RequestId': '41JSV7HTQOVMV2QA6P66I9A7CJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Apr 2023 20:10:38 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '41',
   'connection': 'keep-alive',
   'x-amzn-requestid': '41JSV7HTQOVMV2QA6P66I9A7CJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3073332947'},
  'RetryAttempts': 0}}

In [11]:
#Describe table to see definition of Table
dynamo_client.describe_table(TableName='job_run_details')

{'Table': {'AttributeDefinitions': [{'AttributeName': 'job_id',
    'AttributeType': 'S'},
   {'AttributeName': 'job_run_time', 'AttributeType': 'N'}],
  'TableName': 'job_run_details',
  'KeySchema': [{'AttributeName': 'job_id', 'KeyType': 'HASH'},
   {'AttributeName': 'job_run_time', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 4, 23, 15, 7, 45, 440000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:866874214817:table/job_run_details',
  'TableId': '477c032b-1fd9-4d74-b3d3-e4c0dbbb7e2a',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2023, 4, 23, 15, 7, 45, 440000, tzinfo=tzlocal())}},
 'ResponseMetadata': {'RequestId': '07JNQD6MJI5NRF0BFRUQ9IE4MRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
 

In [12]:
#Create dynamodb resource to insert data into dynamodb table jobs. Using boto3 invoking resource function
dynamodb_resource = boto3.resource('dynamodb')

In [13]:
#check type of  dynamodb_resource
type(dynamodb_resource)

boto3.resources.factory.dynamodb.ServiceResource

In [14]:
#Create table object by using Table constructor which is available on top of ServiceResource within dynamodb
job_details_table = dynamodb_resource.Table('job_run_details')

In [15]:
#validate type of job_detail_table
type(job_details_table)


boto3.resources.factory.dynamodb.Table

In [16]:
import time
import datetime

In [17]:
#Unix representation of time epoch
int(time.mktime(datetime.datetime.now().timetuple()))

1682280807

In [18]:
# create dictionary object which will be used to load the table
jrd_item = {
    'job_id': 'gitaws_ingest',
    'job_run_time': int(time.mktime(datetime.datetime.now().timetuple())),
    'job_run_bookmark_details': {
        'processed_file_name': '2022-06-03.0.json.gz'
    }
}

In [19]:
jrd_item

{'job_id': 'gitaws_ingest',
 'job_run_time': 1682280871,
 'job_run_bookmark_details': {'processed_file_name': '2022-06-03.0.json.gz'}}

In [20]:
#Inserting record into table using put_item and the dictionary created above. Item key word is mandatory for put_item
job_details_table.put_item(
    Item=jrd_item
)

{'ResponseMetadata': {'RequestId': 'C231HOU2C50LVT2U6JUQVOD1INVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Apr 2023 20:14:56 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'C231HOU2C50LVT2U6JUQVOD1INVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [21]:
#returns all the rows in the table
job_details_table.scan()

{'Items': [{'job_id': 'gitaws_ingest',
   'job_run_bookmark_details': {'processed_file_name': '2022-06-03.0.json.gz'},
   'job_run_time': Decimal('1682280871')}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'RMISBT6MGCI7OH8OSBGV7SO69JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Apr 2023 20:15:05 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '191',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'RMISBT6MGCI7OH8OSBGV7SO69JVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '735378482'},
  'RetryAttempts': 0}}

In [ ]:
#get_item returns only one row based on the key passed
job_details_table.get_item(Key={'job_id': 'gitaws_ingest'})

In [ ]:
#Adding additional field into table using dictionary
job_details = {
    'job_id': 'gitaws_ingest',
    'job_description': 'Ingest gharchive data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}

In [ ]:
#overwrites existing item because key is same
job_details_table.put_item(
    Item=job_details
)

In [ ]:
#validate updated data based on the key passed
job_details_table.get_item(Key={'job_id': 'gitaws_ingest'})